# Julia

## Manipulação de dados
### DataFrames

O que veremos nesse tutorial?

1. leitura de dados
2. selecionar linhas
3. selecionar colunas
4. filtro de linhas
5. groupby com estatísticas
6. renomear colunas 
7. dados faltantes


## 1. Leitura de dados

In [ ]:
# importando biblioteca
# using Pkg
# Pkg.add("DataFrames")
# Pkg.add("CSV")
# Pkg.add("Queryverse")
# Pkg.add("BenchmarkTools")
# Pkg.add("Pipe")
using DataFrames
using CSV
using Queryverse
using BenchmarkTools
using Pipe

In [ ]:
# ler os dados
sp500 = CSV.read("data/sp500.csv", DataFrame);

### Mensurando tempo de exeução da leitura dos dados

In [ ]:
@time CSV.read("data/sp500.csv", DataFrame);

In [ ]:
@benchmark df = CSV.read("data/sp500.csv", DataFrame)
# @benchmark CSV.read("data/sp500.csv", DataFrame, tasks=8)

#### Mostrando as 10 primeiras linhas

In [ ]:
first(sp500, 10)

## 2. Selecionar linhas

In [ ]:
## selecionando as linhas 1, 10 e 100
sp500[[1, 10, 100],:]

In [ ]:
## Remover as linhas 1, 10 e 100
sp500[Not([1, 10, 100]), :]

# deleta as linhas inplace
# delete!(sp500, [1, 10, 100])

## 3. Selecionando colunas

In [ ]:
sp500.close
sp500."close"
sp500[!, :close]
sp500[!, "close"]

# se vc quiser fazer uma cópia da coluna
sp500[:, :close] # usar ":" no lugar de "!"

In [ ]:
# mais de uma coluna - preferível 
sp500[!, [:symbol, :close, :volume]]

# alternativa 
sp500[!, ["symbol", "close", "volume"]]

# select
select(sp500, [:symbol, :close, :volume])

# se quiser fazer o inplace, alterar o sp500 use "!"
# select!(sp500, [:symbol, :close, :volume])

In [ ]:
## removendo colunas específicas
sp500[!, Not([:high, :open])]

## select
select(sp500, Not([:high, :open]))

## 4. Selecionando linhas a partir de condições

In [ ]:
## DataFrames 
# diversas maneiras
sp500[sp500.volume .< 1000, :]

filter(:volume => volume -> volume<1000, sp500)

filter(sp500 -> sp500.volume < 1000, sp500)


In [ ]:
sp500[(sp500.volume .< 1000).&(sp500.close .> 60),:]

filter([:volume, :close] => (volume, close) -> volume < 1000 && close > 60, sp500)

#### Usando Queryverse

In [ ]:
sp500 |>
    @filter(_.volume < 1000) |>
    DataFrame

In [ ]:
sp500 |>
    @filter(_.volume < 1000 && _.close > 60) |>
    DataFrame

### Select 

In [ ]:
# Vamos selecionar as colunas que iniciam com v
sp500 |>
    @select(startswith("v")) |>
    DataFrame

In [ ]:
# Não vamos selecionar as colunas que iniciam com v
sp500 |>
    @select(!startswith("v")) |>
    DataFrame

In [ ]:
# Mais de uma condição
sp500 |>
    @select(startswith("c"), startswith("v")) |>
    DataFrame

## 5. Groupby

#### Calculando estatísticas

In [ ]:
using Statistics

In [ ]:
## Descrevendo as estatísticas básicas
describe(sp500)

In [ ]:
# Estatísticas específicas
describe(sp500, :mean, :std)

In [ ]:
#vamos fazer o grupo por symbol e calcular a média do volume
grupos = groupby(sp500, :symbol)
combine(grupos, :volume=>mean)

In [ ]:
@pipe sp500 |>
    groupby(_,:symbol) |>
    combine(_,:volume=>mean)

# alterando nome da coluna
@pipe sp500 |>
    groupby(_,:symbol) |>
    combine(_,:volume=>mean=>:media)

In [ ]:
# vamos fazer o grupo por symbol e calcular a média do volume
sp500 |>
    @groupby(_.symbol) |> 
    @map({
        Key=key(_),
        media=mean(_.volume)}) |>
    DataFrame

In [ ]:
# ordenando de forma crescente
@pipe sp500 |>
    groupby(_,:symbol) |>
    combine(_,:volume=>mean=>:media) |>
    sort(_,:media)

# ordenando de forma decrescente
@pipe sp500 |>
    groupby(_,:symbol) |>
    combine(_,:volume=>mean=>:media) |>
    sort(_,:media, rev=true)

In [ ]:
# ordenando de forma crescente
sp500 |>
    @groupby(_.symbol) |> 
    @map({
        symbol=key(_), 
        media=mean(_.volume)}) |>
    @orderby(_.media) |>
    DataFrame

# ordenando de forma decrescente
sp500 |>
    @groupby(_.symbol) |> 
    @map({
        symbol=key(_), 
        media=mean(_.volume)}) |>
    @orderby_descending(_.media) |>
    DataFrame

In [ ]:
@pipe sp500 |>
    groupby(_,:symbol) |>
    combine(_,:close=>mean,
              :close=>std)

@pipe sp500 |>
    groupby(_,:symbol) |>
    combine(_,:close=>mean=>:media,
              :close=>std=>:desvio)

In [ ]:
estatisticas = sp500 |>
    @groupby(_.symbol) |> 
    @map({Key=key(_), 
          mean=mean(_.close), 
          std=std(_.close)}) |>
    DataFrame 

## 6. Renomear colunas

In [ ]:
estatisticas |>
    @rename(:Key => :ticker, 
            :mean => :media, 
            :std => :desvio) |>
    DataFrame

## 7. Dados faltantes

In [ ]:
# função ismissing retorna bool
# somar a quantidade de dados faltantes para cada coluna
sum_ismiss(df) = sum(ismissing.(df))
[sum_ismiss(col) for col = eachcol(sp500)]

# usando mapcols
mapcols(sum_ismiss, sp500)

In [ ]:
## Missing Data
dropmissing(sp500) # todas as colunas

In [ ]:
# dropando missing apenas de open, high
dropmissing(sp500, :high)